In [56]:
import csv
import pandas as pd

import flywheel
fw = flywheel.Client('', root=True)  # This works if you are logged into a Flywheel instance on a Terminal.
fw.get_config().site.api_url  # Prints the instance you are logged into to make sure it is the right one.

/usr/local/lib/python3.9/site-packages/flywheel/flywheel.py:6274: UserWarning: Client version 15.3.1 does not match server version 14.6.2. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


'https://rollout.ce.flywheel.io/api'

In [100]:
def save_curation_csvs(group_label, project_label):
    """Save BIDS mapping from acquisition names to BIDS paths and fieldmap IntendedFors as sorted csv files."""

    project = fw.projects.find_one(f'group={group_label},label={project_label}')

    intended_for_acquisition = dict()
    intended_for_dirs = dict()
    intended_fors = dict()
    
    nifti_df = pd.DataFrame(columns=("Acquisition label (SeriesDescription)", "File name", "Curated BIDS path"))

    ii = 0  # Current acquisition index
    for acquisition in fw.acquisitions.iter_find(f"project={project.id}"):

        print(ii, "  ", acquisition.label)
        for file in acquisition.reload().files:
            if file.info['BIDS'] != "NA":
                if file.type == 'nifti':
                    if not file.info['BIDS']['ignore']:
                        print(f"{acquisition.label}, {file.name}, {file.info['BIDS']['Folder']}/{file.info['BIDS']['Filename']}")
                        nifti_df.loc[ii] = [acquisition.label, 
                                            file.name, 
                                            f"{file.info['BIDS']['Folder']}/{file.info['BIDS']['Filename']}"]
                    else:
                        print(f"{acquisition.label}, {file.name}, Ignored")
                        nifti_df.loc[ii] = [acquisition.label, file.name, "Ignored"]
                if 'IntendedFor' in file.info['BIDS']:
                    intended_for_acquisition[file.name] = acquisition.label
                    intended_for_dirs[file.name] = file.info['BIDS']['IntendedFor']
                    intended_fors[file.name] = file.info['IntendedFor']
            ii += 1
    
    nifti_df.sort_values(by=["Curated BIDS path"], inplace=True)
    
    nifti_df.to_csv(f'{group_label}_{project_label}_niftis.csv', index=False)  
           
    print()

    with open(f'{group_label}_{project_label}_intendedfors.csv', mode='w') as intendedfors_file:
        intendedfors_writer = csv.writer(intendedfors_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        intendedfors_writer.writerow(["acquisition label", "file name and folder", "IntendedFor List of BIDS paths"])

        for k,v in intended_for_dirs.items():
            print(f"{intended_for_acquisition[k]}, {k}")
            intendedfors_writer.writerow([intended_for_acquisition[k], k])
            for i in v:
                print(f",{i['Folder']}")
                intendedfors_writer.writerow(["",i['Folder']])
                intended_fors[k].sort()
                for j in intended_fors[k]:
                    print(f",,{j}")
                    intendedfors_writer.writerow(["","",j])

    return nifti_df

In [80]:
df = save_curation_csvs('bids-apps','BIDS_multi_session')

0    task-REST_dir-AP_run-1_sbref
task-REST_dir-AP_run-1_sbref, sub-TOME3024_ses-Session1_task-REST_dir-AP_run-1_sbref.nii.gz, func/sub-TOME3024_ses-Session1_task-REST_dir-AP_run-1_sbref.nii.gz
1    task-REST_dir-AP_run-2
task-REST_dir-AP_run-2, sub-TOME3024_ses-Session1_task-REST_dir-AP_run-2_bold.nii.gz, func/sub-TOME3024_ses-Session1_task-REST_dir-AP_run-2_bold.nii.gz
2    acq-MPRHA_T1w
acq-MPRHA_T1w, sub-TOME3024_ses-Session2_acq-MPRHA_T1w.nii.gz, anat/sub-TOME3024_ses-Session2_acq-MPRHA_T1w.nii.gz



In [77]:
df

,Acquisition label (SeriesDescription),File name,Curated BIDS path
2,acq-MPRHA_T1w,sub-TOME3024_ses-Session2_acq-MPRHA_T1w.nii.gz,anat/sub-TOME3024_ses-Session2_acq-MPRHA_T1w.n...
0,task-REST_dir-AP_run-1_sbref,sub-TOME3024_ses-Session1_task-REST_dir-AP_run...,func/sub-TOME3024_ses-Session1_task-REST_dir-A...
1,task-REST_dir-AP_run-2,sub-TOME3024_ses-Session1_task-REST_dir-AP_run...,func/sub-TOME3024_ses-Session1_task-REST_dir-A...
